Tutorial

1. [GEOquery-1](https://l.messenger.com/l.php?u=https%3A%2F%2Fkasperdanielhansen.github.io%2Fgenbioconductor%2Fhtml%2FGEOquery.html&h=AT0PYygqo7DFCuH9uEHTYClGKlOsUXS-QE8-xxwu6Bt4vmjpjHzeVzp8G0cmn849aRvsPk03W85AN8VTiQ5SOvLu8yZbr3dWu14j7f7kFsksDW66ad0STAgqHfm5gCuUb4QcCw)
2. [GEOquery-2](http://genomicsclass.github.io/book/pages/GEOquery.html)
3. [Series Matrix File v.s. Raw Data](https://www.researchgate.net/post/What_do_the_series_matrix_files_signify_in_GEO_data_Where_do_I_get_the_raw_data_for_meaningful_re-analysis_meta_analysis)
4. [RPKM TPM](https://www.rna-seqblog.com/rpkm-fpkm-and-tpm-clearly-explained/)

# A script to get original cell label

- we don't need the count matrix
- only need the metadata

Get information about
- age
- cell type
- sex
- tissue
- and SRP ID (to map to our result)

In [1]:
library(dplyr)
library(tidyr)
library(readxl)
target_directory = '/cellar/users/hsher/Data/ideker/sc/'

# connect to SRA to map SRX to SRR
suppressMessages(library(SRAdb))
srafile = '/cellar/users/hsher/ontoencoder/data/SRAmetadb.sqlite'
sra_con <- dbConnect(SQLite(),srafile)


geo = read_excel('RNA-seq.xlsx')

# datasets to download
geo

suppressMessages(library(GEOquery))


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

New names:
* `` -> ...1


...1,GEO accession,species,SRA
Darmanis,GSE67835,human,SRP057196
Zhang,GSE73721,human,SRP064454
Zhang,GSE52564,mouse,SRP033200
Zeisel,GSE60361,mouse,SRP045452
Tasic,GSE71585,mouse,SRP061902


### not needed: we do not need the processed data anymore
for (geo_id in as.matrix(geo_ids)){
    print(geo_id)
    g_series_matrix <- getGEO(GEO = geo_id, destdir = target_directory)
    g_raw = getGEOSuppFiles(geo_id, baseDir = target_directory)
    
}

# parse metadata for GSE67835

In [2]:
g_series_matrix <- getGEO(GEO = 'GSE67835', destdir = target_directory)

# there are two files
d1 = as.data.frame.matrix(pData(g_series_matrix$`GSE67835-GPL15520_series_matrix.txt.gz`))
d2 = as.data.frame.matrix(pData(g_series_matrix$`GSE67835-GPL18573_series_matrix.txt.gz`))
d<-rbind(d1,d2)

metadata_of_interest = c('geo_accession', 'characteristics_ch1.1', 'characteristics_ch1.2', 'relation.1' ,'supplementary_file_1', 'tissue:ch1')
d <-d[, metadata_of_interest]

d %>% tidyr::separate(characteristics_ch1.1, 
                c("nothing", "cell type"), sep = ': ') ->d
d %>% tidyr::separate(characteristics_ch1.2, 
                c("nothing", "age"), sep = ":") -> d
d %>% tidyr::separate(relation.1,
                c("nothing", "SRX"), sep = '=') -> d

metadata_of_interest = c('cell type', 'age', 'SRX', 'tissue:ch1')
d <-d[, metadata_of_interest]

Found 2 file(s)
GSE67835-GPL15520_series_matrix.txt.gz
Using locally cached version: /cellar/users/hsher/Data/ideker/sc//GSE67835-GPL15520_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.
Using locally cached version of GPL15520 found here:
/cellar/users/hsher/Data/ideker/sc//GPL15520.soft 
GSE67835-GPL18573_series_matrix.txt.gz
Using locally cached version: /cellar/users/hsher/Data/ideker/sc//GSE67835-GPL18573_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.
Using locally cached version of GPL18573 found here:
/cellar/users/hsher/Data/ideker/sc//GPL18573.soft 


In [3]:

colnames(d) <- c('cell type', 'age', 'SRX', 'tissue')
d[,'GSM'] <- rownames(d)
head(d)

,cell type,age,SRX,tissue,GSM
GSM1657992,astrocytes,postnatal 47 years,SRX995982,cortex,GSM1657992
GSM1657993,endothelial,postnatal 47 years,SRX995983,cortex,GSM1657993
GSM1657994,microglia,postnatal 47 years,SRX995984,cortex,GSM1657994
GSM1657995,endothelial,postnatal 47 years,SRX995985,cortex,GSM1657995
GSM1657996,microglia,postnatal 47 years,SRX995986,cortex,GSM1657996
GSM1657997,microglia,postnatal 63 years,SRX995987,cortex,GSM1657997


In [4]:
# map SRX to SRR #SRP057196
mapper <- getFASTQinfo('SRP057196',sra_con)[,c('run', 'experiment')]
mapper <- mapper[!duplicated(mapper),]
merged_d<- merge(mapper, d, by.x = 'experiment', by.y = 'SRX')


In [5]:
head(merged_d)

experiment,run,cell type,age,tissue,GSM
SRX995861,SRR1974543,oligodendrocytes,postnatal 54 years,cortex,GSM1657871
SRX995862,SRR1974544,hybrid,postnatal 54 years,cortex,GSM1657872
SRX995863,SRR1974545,oligodendrocytes,postnatal 54 years,cortex,GSM1657873
SRX995864,SRR1974546,hybrid,postnatal 54 years,cortex,GSM1657874
SRX995865,SRR1974547,hybrid,postnatal 54 years,cortex,GSM1657875
SRX995866,SRR1974548,oligodendrocytes,postnatal 54 years,cortex,GSM1657876


In [6]:
merged_d %>% write.csv('GSE67835_metadata.csv')

# GSE73721

In [7]:
g_series_matrix <- getGEO(GEO = 'GSE73721', destdir = target_directory)
d1 = as.data.frame.matrix(pData(g_series_matrix$'GSE73721-GPL18573_series_matrix.txt.gz'))
d2 = as.data.frame.matrix(pData(g_series_matrix$'GSE73721-GPL19057_series_matrix.txt.gz'))
d<- rbind(d1, d2)

metadata_of_interest = c('cell type:ch1', 'relation.1', 'source_name_ch1', 'age:ch1')
d <-d[, metadata_of_interest]

d %>% tidyr::separate(relation.1,
                c("nothing", "SRX"), sep = '=') -> d
d %>% tidyr::separate(source_name_ch1, 
                c("maturity", "tissue"), sep = ' ') ->d
#d %>% tidyr::separate(age:ch1, 
#                c("nothing", "age"), sep = ":") -> d

metadata_of_interest = c('cell type:ch1','SRX','maturity','tissue', 'age:ch1' )
d <- d[, metadata_of_interest]
colnames(d) <- c('cell type','SRX','maturity','tissue', 'age' )

Found 2 file(s)
GSE73721-GPL18573_series_matrix.txt.gz
Using locally cached version: /cellar/users/hsher/Data/ideker/sc//GSE73721-GPL18573_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.
Using locally cached version of GPL18573 found here:
/cellar/users/hsher/Data/ideker/sc//GPL18573.soft 
GSE73721-GPL19057_series_matrix.txt.gz
Using locally cached version: /cellar/users/hsher/Data/ideker/sc//GSE73721-GPL19057_series_matrix.txt.gz
Parsed with column specification:
cols(
  ID_REF = col_character(),
  GSM1901329 = col_character(),
  GSM1901330 = col_character(),
  GSM1901331 = col_character(),
  GSM1901332 = col_character()
)
Using locally cached version of GPL19057 found here:
/cellar/users/hsher/Data/ideker/sc//GPL19057.soft 
Warning message:
“Expected 2 pieces. Additional pieces discarded in 31 rows [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 27, 28, 29, 30, ...].”Warning messag

In [8]:
d[, 'GSM']<- rownames(d)
head(d)

,cell type,SRX,maturity,tissue,age,GSM
GSM1901303,Astrocyte,SRX1303782,Fetal,cortex,18 gestational weeks,GSM1901303
GSM1901304,Astrocyte,SRX1303783,Fetal,cortex,18 gestational weeks,GSM1901304
GSM1901305,Astrocyte,SRX1303784,Fetal,cortex,18.5 gestational weeks,GSM1901305
GSM1901306,Astrocyte,SRX1303785,Fetal,cortex,18.1 gestational weeks,GSM1901306
GSM1901307,Astrocyte,SRX1303786,Fetal,cortex,18.5 gestational weeks,GSM1901307
GSM1901308,Astrocyte,SRX1303787,Fetal,cortex,18.5 gestational weeks,GSM1901308


In [9]:
mapper <- getFASTQinfo('SRP064454',sra_con)[,c('run', 'experiment')]
mapper <- mapper[!duplicated(mapper),]
merged_d<- merge(mapper, d, by.x = 'experiment', by.y = 'SRX')

In [10]:
head(merged_d)

experiment,run,cell type,maturity,tissue,age,GSM
SRX1303782,SRR2557083,Astrocyte,Fetal,cortex,18 gestational weeks,GSM1901303
SRX1303783,SRR2557084,Astrocyte,Fetal,cortex,18 gestational weeks,GSM1901304
SRX1303784,SRR2557085,Astrocyte,Fetal,cortex,18.5 gestational weeks,GSM1901305
SRX1303785,SRR2557086,Astrocyte,Fetal,cortex,18.1 gestational weeks,GSM1901306
SRX1303786,SRR2557087,Astrocyte,Fetal,cortex,18.5 gestational weeks,GSM1901307
SRX1303787,SRR2557088,Astrocyte,Fetal,cortex,18.5 gestational weeks,GSM1901308


In [11]:
merged_d %>% write.csv('GSE73721_metadata.csv')

# GSE52564

In [12]:
g_series_matrix <- getGEO(GEO = 'GSE52564', destdir = target_directory)
d <- as.data.frame.matrix(pData(g_series_matrix$GSE52564_series_matrix.txt.gz))
metadata_of_interest <- c('tissue:ch1', 'title', 'relation.2', 'relation.3')
d <- d[, metadata_of_interest]

Found 1 file(s)
GSE52564_series_matrix.txt.gz
Using locally cached version: /cellar/users/hsher/Data/ideker/sc//GSE52564_series_matrix.txt.gz
Parsed with column specification:
cols(
  ID_REF = col_character(),
  GSM1269903 = col_character(),
  GSM1269904 = col_character(),
  GSM1269905 = col_character(),
  GSM1269906 = col_character(),
  GSM1269907 = col_character(),
  GSM1269908 = col_character(),
  GSM1269909 = col_character(),
  GSM1269910 = col_character(),
  GSM1269911 = col_character(),
  GSM1269912 = col_character(),
  GSM1269913 = col_character(),
  GSM1269914 = col_character(),
  GSM1269915 = col_character(),
  GSM1269916 = col_character(),
  GSM1269917 = col_character(),
  GSM1269918 = col_character(),
  GSM1269919 = col_character()
)
Using locally cached version of GPL13112 found here:
/cellar/users/hsher/Data/ideker/sc//GPL13112.soft 


In [13]:
d

,tissue:ch1,title,relation.2,relation.3
GSM1269903,cerebral cortex,Astrocyte1,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN02415123,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX380379
GSM1269904,cerebral cortex,Astrocyte2,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN02415122,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX380380
GSM1269905,cerebral cortex,Neuron1,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN02415125,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX380381
GSM1269906,cerebral cortex,Neuron2,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN02415124,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX380382
GSM1269907,cerebral cortex,OPC1,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX380383,
GSM1269908,cerebral cortex,OPC2,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX380384,
GSM1269909,cerebral cortex,NFO1,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX380385,
GSM1269910,cerebral cortex,NFO2,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX380386,
GSM1269911,cerebral cortex,MO1,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN02415132,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX380387
GSM1269912,cerebral cortex,MO2,BioSample: https://www.ncbi.nlm.nih.gov/biosample/SAMN02415129,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX380388


In [14]:
g_series_matrix <- getGEO(GEO = 'GSE52564', destdir = target_directory)
d <- as.data.frame.matrix(pData(g_series_matrix$GSE52564_series_matrix.txt.gz))
metadata_of_interest <- c('tissue:ch1', 'title', 'relation.2', 'relation.3')
d <- d[, metadata_of_interest]
d[, 'GSM'] <- rownames(d)

d %>% tidyr::separate(relation.2,
                c("nothing", "SRX"), sep = '=') -> d
d %>% tidyr::separate(relation.3, 
                c("nothing", "SRX_2"), sep = '=') ->d
#d %>% tidyr::separate(age:ch1, 

d %>% mutate(SRXXX = coalesce(SRX, SRX_2)) -> d

metadata_of_interest <- c('tissue:ch1', 'title', 'SRXXX', 'GSM')
d <- d[, metadata_of_interest]
colnames(d) <- c('tissue', 'cell type', 'SRX', 'GSM')

Found 1 file(s)
GSE52564_series_matrix.txt.gz
Using locally cached version: /cellar/users/hsher/Data/ideker/sc//GSE52564_series_matrix.txt.gz
Parsed with column specification:
cols(
  ID_REF = col_character(),
  GSM1269903 = col_character(),
  GSM1269904 = col_character(),
  GSM1269905 = col_character(),
  GSM1269906 = col_character(),
  GSM1269907 = col_character(),
  GSM1269908 = col_character(),
  GSM1269909 = col_character(),
  GSM1269910 = col_character(),
  GSM1269911 = col_character(),
  GSM1269912 = col_character(),
  GSM1269913 = col_character(),
  GSM1269914 = col_character(),
  GSM1269915 = col_character(),
  GSM1269916 = col_character(),
  GSM1269917 = col_character(),
  GSM1269918 = col_character(),
  GSM1269919 = col_character()
)
Using locally cached version of GPL13112 found here:
/cellar/users/hsher/Data/ideker/sc//GPL13112.soft 
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 10 rows [1, 2, 3, 4, 9, 10, 11, 12, 13, 14].”Warning message:
“Expect

In [15]:
# map SRX to SRR #SRP057196
mapper <- getFASTQinfo('SRP033200',sra_con)[,c('run', 'experiment')]
mapper <- mapper[!duplicated(mapper),]
merged_d<- merge(mapper, d, by.x = 'experiment', by.y = 'SRX')


In [16]:
head(merged_d)

experiment,run,tissue,cell type,GSM
SRX380379,SRR1033783,cerebral cortex,Astrocyte1,GSM1269903
SRX380380,SRR1033784,cerebral cortex,Astrocyte2,GSM1269904
SRX380381,SRR1033785,cerebral cortex,Neuron1,GSM1269905
SRX380382,SRR1033786,cerebral cortex,Neuron2,GSM1269906
SRX380383,SRR1033787,cerebral cortex,OPC1,GSM1269907
SRX380384,SRR1033788,cerebral cortex,OPC2,GSM1269908


In [17]:
d %>% write.csv('GSE52564_metadata.csv')

# GSE 60361

In [18]:
g_series_matrix <- getGEO(GEO = 'GSE60361', destdir = target_directory)

d <- as.data.frame.matrix(pData(g_series_matrix$GSE60361_series_matrix.txt.gz))
metadata_of_interest <- c('relation.1', 'age:ch1', 'Sex:ch1')
d <- d[, metadata_of_interest]

d %>% tidyr::separate(relation.1,
                c("nothing", "SRX"), sep = '=') -> d
d[,c('SRX', 'age:ch1', 'Sex:ch1')] -> d
colnames(d) <- c('SRX', 'age', 'sex')
d[, 'GSM'] <- rownames(d)

Found 1 file(s)
GSE60361_series_matrix.txt.gz
Using locally cached version: /cellar/users/hsher/Data/ideker/sc//GSE60361_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.
Using locally cached version of GPL13112 found here:
/cellar/users/hsher/Data/ideker/sc//GPL13112.soft 


In [19]:
# map SRX to SRR #SRP057196
mapper <- getFASTQinfo('SRP045452',sra_con)[,c('run', 'experiment')]
mapper <- mapper[!duplicated(mapper),]
merged_d<- merge(mapper, d, by.x = 'experiment', by.y = 'SRX')


In [20]:
head(merged_d)

experiment,run,age,sex,GSM
SRX676062,SRR1544693,P26,female,GSM1474251
SRX676063,SRR1544694,P26,female,GSM1474252
SRX676064,SRR1544695,P26,female,GSM1474253
SRX676065,SRR1544696,P26,female,GSM1474254
SRX676066,SRR1544697,P26,female,GSM1474255
SRX676067,SRR1544698,P26,female,GSM1474256


In [21]:
merged_d %>% write.csv('GSE60361_metadata.csv')

# GSE71585

In [22]:
g_series_matrix <- getGEO(GEO = 'GSE71585', destdir = target_directory)
d1 <- as.data.frame.matrix(pData(g_series_matrix$`GSE71585-GPL13112_series_matrix.txt.gz`))
d2 <- as.data.frame.matrix(pData(g_series_matrix$`GSE71585-GPL16417_series_matrix.txt.gz`))
d3 <- as.data.frame.matrix(pData(g_series_matrix$`GSE71585-GPL17021_series_matrix.txt.gz`))
metadata_of_interest <- c('source_name_ch1', 'characteristics_ch1', 'relation.1', 'characteristics_ch1.8')
d <- rbind(d1[, metadata_of_interest],d2[, metadata_of_interest],d3[, metadata_of_interest])
d %>% tidyr::separate(relation.1,
                c("nothing", "SRX"), sep = '=') -> d
d %>% tidyr::separate(characteristics_ch1.8,
                c("nothing", "cell type"), sep = ':') -> d
metadata_of_interest <- c('source_name_ch1', 'SRX', 'cell type')
d <- d[,metadata_of_interest]
colnames(d) <-  c('tissue', 'SRX', 'cell type')
d[,'GSM'] <- rownames(d)


Found 3 file(s)
GSE71585-GPL13112_series_matrix.txt.gz
Using locally cached version: /cellar/users/hsher/Data/ideker/sc//GSE71585-GPL13112_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.
Using locally cached version of GPL13112 found here:
/cellar/users/hsher/Data/ideker/sc//GPL13112.soft 
GSE71585-GPL16417_series_matrix.txt.gz
Using locally cached version: /cellar/users/hsher/Data/ideker/sc//GSE71585-GPL16417_series_matrix.txt.gz
Parsed with column specification:
cols(
  ID_REF = col_character(),
  GSM1839769 = col_character()
)
Using locally cached version of GPL16417 found here:
/cellar/users/hsher/Data/ideker/sc//GPL16417.soft 
GSE71585-GPL17021_series_matrix.txt.gz
Using locally cached version: /cellar/users/hsher/Data/ideker/sc//GSE71585-GPL17021_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.
Using loc

In [23]:
# map SRX to SRR #SRP057196
mapper <- getFASTQinfo('SRP061902',sra_con)[,c('run', 'experiment')]
mapper <- mapper[!duplicated(mapper),]
merged_d<- merge(mapper, d, by.x = 'experiment', by.y = 'SRX')


In [24]:
head(merged_d)

experiment,run,tissue,cell type,GSM
SRX1127668,SRR2138604,Primary visual cortex,Vip Mybpc1,GSM1839192
SRX1127669,SRR2138605,Primary visual cortex,Vip Cxcl14_Car4,GSM1839193
SRX1127670,SRR2138606,Primary visual cortex,L4 Sparcl1,GSM1839194
SRX1127671,SRR2138607,Primary visual cortex,Vip Chat,GSM1839195
SRX1127672,SRR2138608,Primary visual cortex,Vip Cxcl14_Car4,GSM1839196
SRX1127673,SRR2138609,Primary visual cortex,L2/3 Otof,GSM1839197


In [25]:
merged_d %>% write.csv('GSE71585_metadata.csv')